In [1]:
import pandas as pd
import numpy as np
#import pymorphy2
from collections import Counter, defaultdict
import utils
import sys
from index import Index
import index
import score
from multiprocessing import Pool
from itertools import repeat

In [2]:
from importlib import reload
#index = reload(index)
#Index = index.Index
score = reload(score)

In [3]:
index_full = Index('data/docs.tsv', norm_method = 'none', debug=1)

In [8]:
index_full = utils.load_from_file('pkls/bm25_unigram.pkl')
#index_full = utils.copy_model(index_full, Index)

In [ ]:
#index_full.build()

In [12]:
#utils.save_to_file(index_full, "pkls/bm25_unigram.pkl")

In [9]:
queries = utils.load_queries('data/queries_ec.tsv')
test_data = utils.load_test('data/test.tsv')
train_data = utils.load_train('data/clicks.train.tsv')

In [13]:
def score_internal(data, ind, docs_file_name, index_file_name):
        def score_qd(doc_wrap, query, doc_id):
            title, body = doc_wrap.get(doc_id)
            return score.score_bm25(ind, query, title, body)
    
        doc_wrap = utils.DocWrap(docs_file_name, index_file_name)
        results = []
        for query_id, docs in data:
            query = queries[query_id]
            for doc_id in docs:
                title_score, body_score, whole_score = score_qd(doc_wrap, query, doc_id)
                results.append((query_id, doc_id, title_score, body_score, whole_score))
        print("Processed {} query, doc pairs".format(len(data)))
        return pd.DataFrame(results,columns=['query_id', 'doc_id', 'score_title', 'score_body', 'score_whole'])

def score_documents(ind, queries, test_data, train_data, docs_file_name, index_file_name, idf_smooth=0.5):    
    def chunks(data, num_chunks=24):
        n = len(data)//num_chunks + 1
        return [data[i:i+n] for i in range(0, len(data), n)]
    
    print("Calc idfs")
    ind.calc_idfs(idf_smooth)
    
    print("Processing train, {} pairs".format(len(train_data)))
    tr_data = [(x,y) for x,y,_ in train_data]
    pool = Pool()
    temp = pool.starmap(score_internal, zip(chunks(tr_data),
                                              repeat(ind),
                                              repeat(docs_file_name),
                                              repeat(index_file_name)))
    train_results = pd.concat(temp, ignore_index=True)
    
    print("Processing test, {} pairs".format(len(test_data)))
    temp = pool.starmap(score_internal, zip(chunks(test_data),
                                              repeat(ind),
                                              repeat(docs_file_name),
                                              repeat(index_file_name)))
    test_results = pd.concat(temp, ignore_index=True)
    
    pool.close()
    pool.join()
    pool.terminate()
    
    return train_results, test_results

In [11]:
def save_pkls(train_results, test_results, columns, file_name):
    train_results.columns = columns
    test_results.columns = columns
    utils.save_to_file(train_results, file_name.format('train'))
    utils.save_to_file(test_results, file_name.format('test'))

In [12]:
train_results, test_results = score_documents(index_full,
                                              queries,
                                              test_data,
                                              train_data,
                                              'data/docs.tsv',
                                              'pkls/index.pkl',
                                              idf_smooth=0.5)

Calc idfs
Processing train, 3309 pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 135 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processing test, 816 pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 

In [17]:
columns=['query_id', 'doc_id', 'score_title_bm25_smooth', 'score_body_bm25_smooth', 'score_whole_bm25_smooth']
save_pkls(train_results, test_results, columns, "pkls/df_bm25_unigram_scores_smooth_{}.pkl")

### Нормированные тексты

In [18]:
index_full = utils.load_from_file('pkls/bm25_unigram_norm.pkl')
#index_full = utils.copy_model(index_full, Index)

In [21]:
#utils.save_to_file(index_full, "pkls/bm25_unigram_norm.pkl")

In [20]:
queries = utils.load_queries('data/queries_norm.tsv')

In [22]:
train_results, test_results = score_documents(index_full,
                                              queries,
                                              test_data,
                                              train_data,
                                              'data/docs_normed2.tsv',
                                              'pkls/index_norm.pkl',
                                              idf_smooth=0.5)

Calc idfs
Processing train, 3309 pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 135 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processing test, 816 pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 

Process ForkPoolWorker-30:
Process ForkPoolWorker-22:
Process ForkPoolWorker-26:
Process ForkPoolWorker-32:
Process ForkPoolWorker-29:
Process ForkPoolWorker-27:
Traceback (most recent call last):
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process ForkPoolWorker-25:
Process ForkPoolWorker-31:
Process ForkPoolWorker-17:
Process ForkPoolWorker-18:
Process ForkPoolWorker-23:
Process ForkPoolWorker-21:
Process ForkPoolWorker-19:
Process ForkPoolWorker-28:
Process ForkPoolWorker-20:
Process ForkPoolWorker-24:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/centos/tools/anaconda3/lib/python3.6

  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = 

In [23]:
columns=['query_id', 'doc_id', 'score_title_bm25_smooth_norm',
         'score_body_bm25_smooth_norm', 'score_whole_bm25_smooth_norm']
save_pkls(train_results, test_results, columns, "pkls/df_bm25_unigram_scores_smooth_norm_{}.pkl")

### Биграммы

In [24]:
index_full = utils.load_from_file('pkls/bm25_bigrams.pkl')
index_full = utils.copy_model(index_full, Index)

In [25]:
utils.save_to_file(index_full, "pkls/bm25_bigrams.pkl")

In [ ]:
queries = utils.load_queries('data/queries_ec.tsv')

In [ ]:
train_results, test_results = score_documents(index_full,
                                              queries,
                                              test_data,
                                              train_data,
                                              'data/docs.tsv',
                                              'pkls/index.pkl',
                                              idf_smooth=0.5)

Calc idfs
Processing train, 3309 pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 135 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processing test, 816 pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 

Process ForkPoolWorker-38:
Process ForkPoolWorker-36:
Process ForkPoolWorker-47:
Process ForkPoolWorker-48:
Process ForkPoolWorker-34:
Process ForkPoolWorker-41:
Process ForkPoolWorker-45:
Process ForkPoolWorker-39:
Process ForkPoolWorker-44:
Process ForkPoolWorker-46:
Process ForkPoolWorker-43:
Process ForkPoolWorker-35:
Process ForkPoolWorker-42:
Process ForkPoolWorker-40:
Process ForkPoolWorker-33:
Process ForkPoolWorker-37:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most

  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/centos/tools/anaconda3/lib/python3.6/multiprocessing/queues.py", line 342, i

In [ ]:
columns=['query_id', 'doc_id', 'score_title_bm25_bigram_smooth',
         'score_body_bm25_bigram_smooth', 'score_whole_bm25_bigram_smooth']
save_pkls(train_results, test_results, columns, "pkls/df_bm25_bigram_scores_smooth_{}.pkl")

### Биграммы норм

In [14]:
index_full = utils.load_from_file('pkls/bm25_bigrams_norm.pkl')
#index_full = utils.copy_model(index_full, Index)

In [15]:
#utils.save_to_file(index_full, "pkls/bm25_bigrams_norm.pkl")

In [16]:
queries = utils.load_queries('data/queries_norm.tsv')

In [17]:
train_results, test_results = score_documents(index_full,
                                              queries,
                                              test_data,
                                              train_data,
                                              'data/docs_normed2.tsv',
                                              'pkls/index_norm.pkl',
                                              idf_smooth=0.5)

Calc idfs
Processing train, 3309 pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 135 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processed 138 query, doc pairs
Processing test, 816 pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 35 query, doc pairs
Processed 

In [18]:
columns=['query_id', 'doc_id', 'score_title_bm25_bigram_smooth_norm',
         'score_body_bm25_bigram_smooth_norm', 'score_whole_bm25_bigram_smooth_norm']
save_pkls(train_results, test_results, columns, "pkls/df_bm25_bigram_scores_smooth_{}.pkl")

### Триграммы

In [19]:
index_full = utils.load_from_file('pkls/bm25_char_trigrams.pkl')
index_full = utils.copy_model(index_full, Index)

In [20]:
utils.save_to_file(index_full, "pkls/bm25_char_trigrams.pkl")

In [ ]:
queries = utils.load_queries('data/queries_ec.tsv')

In [ ]:
train_results, test_results = score_documents(index_full,
                                              queries,
                                              test_data,
                                              train_data,
                                              'data/docs.tsv',
                                              'pkls/index.pkl',
                                              idf_smooth=0.5)

Calc idfs
Processing train, 3309 pairs


In [ ]:
columns=['query_id', 'doc_id', 'score_title_bm25_char_trigram_smooth',
         'score_body_bm25_char_trigram_smooth', 'score_whole_bm25_char_trigram_smooth']
save_pkls(train_results, test_results, columns, "pkls/df_bm25_char_trigram_scores_smooth_{}.pkl")